# sqlite3


Scripts to run database migration, table creation etc.

In [1]:
import sqlite3
import json

In [2]:
class dbopen(object):
    def __init__(self, path='data.db'):
        self.path = path
        
    def __enter__(self):
        self.conn = sqlite3.connect(self.path)
        self.cursor = self.conn.cursor()
        return self.cursor
    
    def __exit__(self, exc_class, exc, traceback):
        self.conn.commit()
        self.conn.close()

In [3]:
def create_table():
    with dbopen() as c:
        c.execute("""
        CREATE TABLE IF NOT EXISTS skill (
            id integer PRIMARY KEY AUTOINCREMENT,
            url text UNIQUE NOT NULL,
            text text NOT NULL,
            data json NOT NULL,
            created_at timestamp NOT NULL DEFAULT CURRENT_TIMESTAMP,
            updated_at timestamp NOT NULL DEFAULT CURRENT_TIMESTAMP
        )""")

        c.execute("""
        CREATE TRIGGER IF NOT EXISTS update_timestamp
        AFTER UPDATE ON skill 
        BEGIN
            update skill set updated_at = current_timestamp WHERE url = NEW.url;
        END
        """)

In [4]:
with dbopen('data.db') as c:
    result = c.execute('select id, url from skill').fetchall()
    print(result)

[(3, 'https://buttercms.com/blog/vue-vs-react-which-is-the-better-framework'), (1, 'https://www.fullstackacademy.com/blog/nine-best-programming-languages-to-learn'), (2, 'https://www.ignite.digital/10-best-programming-languages-to-learn-in-2020/')]


# Preparing Training Data

In order to train the data for Spacy's NER, we first need to convert the format to the training data's format. We will load the data that is already stored in the database and change it to the following format:

```python
train_data = [
    ('React, Vue is good', {"entities": [(0, 5, 'SKILL'), (7, 10, 'SKILL')]})
]
```


References:
- https://spacy.io/usage/training
- https://www.machinelearningplus.com/nlp/training-custom-ner-model-in-spacy/

In [5]:
train_data = []
with dbopen() as c:
    result = c.execute('select id, data from skill').fetchall()
    for id, data in result:
        json_data = json.loads(data)
        for row in json_data:
            txt, annotations = row['text'], row['annotations']
            
            annotation_format = [(annotation['start'], 
                                  annotation['end'], 
                                  annotation['label']) for annotation in annotations]
            item_format = (txt, {"entities": annotation_format})
            train_data.append(item_format)

In [8]:
len(train_data)

242

# Updating the Named Entity Recognizer

In [9]:
import spacy

# Load pre-existing spacy model.
nlp = spacy.load('en_core_web_sm')

# Getting the pipeline component.
ner = nlp.get_pipe('ner')

In [10]:
# Adding labels to the `ner`.
for _, annotations in train_data:
    for ent in annotations.get('entities'):
        ner.add_label(ent[2])

In [11]:
# Apart from `ner`, the model has other pipeline components. 
# These components should not get affected in training.
# Disable the other pipeline components that you do not want to change.
# Training will be performed with the unaffected_pipes disabled

pipe_exceptions = ['ner', 'trf_wordpiecer', 'trf_tok2vec']
unaffected_pipes = [pipe for pipe in nlp.pipe_names if pipe not in pipe_exceptions]
unaffected_pipes

['tagger', 'parser']

In [12]:
import random
from spacy.util import minibatch, compounding
from pathlib import Path

In [34]:
%time
# Training the model.
with nlp.disable_pipes(*unaffected_pipes):
    

    for iteration in range(30): # 1
        random.shuffle(train_data) # 2
        losses = {}
        
        batches = minibatch(train_data, 
                            size=compounding(4.0, 32.0, 1.001)) # 3
            
        for batch in batches:
            texts, annotations = zip(*batch)
            nlp.update(texts,
                       annotations,
                       drop=0.5, # Make it harder to memorize data.
                       losses=losses)
        print('Iteration:', iteration, 'Losses:', losses)
print('Completed')

Losses: {'ner': 4313.194564044476}
Losses: {'ner': 4319.868515312672}
Losses: {'ner': 4250.519708633423}
Losses: {'ner': 4297.394887566334}
Losses: {'ner': 4266.092610955238}
Losses: {'ner': 4257.727110803127}
Losses: {'ner': 4242.515068352222}
Losses: {'ner': 4259.796709120274}
Losses: {'ner': 4232.821398139}
Losses: {'ner': 4288.099318742752}
Losses: {'ner': 4242.986487769755}
Losses: {'ner': 4280.384516149759}
Losses: {'ner': 4192.164559528232}
Losses: {'ner': 4227.983571827412}
Losses: {'ner': 4322.200461566448}
Losses: {'ner': 4258.277119457722}
Losses: {'ner': 4324.17472076416}
Losses: {'ner': 4188.70452439785}
Losses: {'ner': 4281.788178563118}
Losses: {'ner': 4181.140057787299}
Losses: {'ner': 4192.5527376532555}
Losses: {'ner': 4245.895878076553}
Losses: {'ner': 4241.292424976826}
Losses: {'ner': 4230.300799235702}
Losses: {'ner': 4249.96360481903}
Losses: {'ner': 4227.023162007332}
Losses: {'ner': 4224.95201574266}
Losses: {'ner': 4202.549118487164}
Losses: {'ner': 4229.39578

### Notes

1. To train the `ner` model, the model has to be looped over the example for sufficient number of iterations. If you train it for like just 5-6 iterations, it may not be effective. NOTE: find out what is the most effective number.
2. Before every iteration it's a good practice to shuffle the examples randomly through random.shuffle() function. 
This ensures that the model does not make generalization based on the order of the examples.
3. Training data is passed in batch. We use `minibatch()` function over the training data that will return data in batches. 

# Testing the model

In [14]:
doc = nlp('I want to learn Scala, go is also good.')
print('Entities', [(ent.text, ent.label_) 
                  for ent in doc.ents])

Entities [('Scala', 'SKILL'), ('go', 'SKILL')]


In [15]:
doc = nlp('can I learn js? Using a single threaded language like node.js is good for IO')
print('Entities', [(ent.text, ent.label_) 
                  for ent in doc.ents])

Entities [('node.js', 'SKILL')]


# Saving the model

In [16]:
from datetime import datetime
today = datetime.now().strftime('%Y_%m_%d')
today

'2020_08_07'

In [17]:
import os
target_dir = f'content/{today}/'

if not os.path.exists(target_dir):
    os.makedirs(target_dir)
output_dir = Path(target_dir)
nlp.to_disk(output_dir)
print('Saved model to', output_dir)

Saved model to content/2020_08_07


# Loading the model and predict

In [18]:
print('Loading from', target_dir)

Loading from content/2020_08_07/


In [33]:
nlp_updated = spacy.load(output_dir)

test_data = ['Kotlin is an awesome programming language', 
             'Should I learn ReactJS in 2020?',
             'Peter is learning Python, Go, JavaScript and Rust',
             'Apple uses Swift programming language for iOS']
for data in test_data:
    doc = nlp_updated(data)
    print('Entities', [(ent.text, ent.label_) for ent in doc.ents])

Entities [('Kotlin', 'SKILL')]
Entities [('ReactJS', 'SKILL')]
Entities [('Python', 'SKILL'), ('Go', 'SKILL'), ('JavaScript', 'SKILL'), ('Rust', 'SKILL')]
Entities [('Swift', 'SKILL')]
